In [35]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras import models
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import os
from tensorflow.keras.applications import ResNet101V2, ResNet50, ResNet101, ResNet152, ResNet50V2, DenseNet121, DenseNet201, ResNet152V2, VGG16, NASNetMobile, EfficientNetB3, VGG19, DenseNet169
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import pathlib
from keras.layers import Input, Dense, BatchNormalization, Dropout, GaussianNoise, GlobalAveragePooling2D
from keras import Sequential
from keras import regularizers

In [54]:
# spine_dir = 'D:/cervical fracture/'
# list_fracture = list(os.listdir(spine_dir+'fracture/'))
# list_normal = list(os.listdir(spine_dir+'normal/'))

In [43]:
# for img in list_fracture:
#     file_name = spine_dir + 'fracture/' + img
#     np_image = Image.open(file_name).convert('L')

#     np_image = np_image.resize((224,224), Image.ANTIALIAS)
#     np_image = np.array(np_image)
#     im = Image.fromarray(np_image)
#     im.save('C:/Users/Cat/clean/fracture/'+img, format='JPEG', quality=100)

C:\Users\Cat\AppData\Local\Temp\ipykernel_34632\3321187752.py:5: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  np_image = np_image.resize((224,224), Image.ANTIALIAS)


In [44]:
# for img in list_normal:
#     file_name = spine_dir + 'normal/' + img
#     np_image = Image.open(file_name).convert('L')

#     np_image = np_image.resize((224,224), Image.ANTIALIAS)
#     np_image = np.array(np_image)
#     im = Image.fromarray(np_image)
#     im.save('C:/Users/Cat/clean/normal/'+img, format='JPEG', quality=100)

C:\Users\Cat\AppData\Local\Temp\ipykernel_34632\1343093081.py:5: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  np_image = np_image.resize((224,224), Image.ANTIALIAS)


In [45]:
spine_dir = 'D:/clean/'
spine_dir = pathlib.Path(spine_dir)

spine_image_dict = {
    'fracture': list(spine_dir.glob('fracture/*')),
    'normal': list(spine_dir.glob('normal/*'))
}
spine_lables_dict = {
    'fracture': 0,
    'normal': 1
}

In [46]:
X, y = [], []
for spine_name, images in spine_image_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        X.append(img)
        y.append(spine_lables_dict[spine_name])

In [47]:
X = np.array(X)
y = np.array(y)

In [48]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train1, X_val, y_train1, y_val = train_test_split(X_train, y_train, test_size=0.2) 

X_train1 = X_train1 / 255.0
X_test = X_test / 255.0
X_val = X_val / 255.0

print(X_train1.shape, X_test.shape, y_train1.shape, y_test.shape, X_val.shape, y_val.shape)

(3024, 224, 224, 3) (420, 224, 224, 3) (3024,) (420,) (756, 224, 224, 3) (756,)


In [51]:
inputs = X_train1.astype('float32')
targets = y_train1

X_val = X_val.astype('float32')

X_test2 = X_test.astype('float32')
y_test2 = y_test

model_history=[]
cnn_models = [
            DenseNet201(include_top=True, weights=None, input_shape=(75, 75, 3), pooling='avg', classes=2)
]

In [ ]:
model = Sequential()
model.add(cnn_models[0])
model.add(Dense(64, kernel_regularizer=regularizers.l2(0.0001), activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, kernel_regularizer=regularizers.l2(0.0001), activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(16, kernel_regularizer=regularizers.l2(0.0001), activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

In [52]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(inputs, targets,
                    validation_data=(X_val, y_val),
                    epochs=100,
                    batch_size=190
                    )
plt.figure(figsize=(20, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], linestyle='solid', color='C0' )
plt.plot(history.history['val_accuracy'], linestyle='dashed', color='C1')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['train_accuracy', 'validation_accuracy'], loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], linestyle='solid', color='C0')
plt.plot(history.history['val_loss'], linestyle='dashed', color='C1')
plt.show()
model_history.append(history.history)
y_pred = model.predict(X_test)
print(model.evaluate(X_test, y_test))
y_preds = model.predict(X_test)
y_preds = np.round(y_preds)
print("Classification Report: \n", classification_report(y_test, y_preds))

Epoch 1/50
12/95 [==>...........................] - ETA: 11:24 - loss: 0.6947 - accuracy: 0.4775 - f1_m: 0.2596 - precision_m: 0.2643 - recall_m: 0.3703

KeyboardInterrupt: 